In [1]:
import pandas as pd
import numpy as np

# Import libraries for tools
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder,OneHotEncoder,LabelEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split

# pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Model Training 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,f1_score,recall_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier

# For hyper Parameter tunning
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

C:\Users\aakas\AppData\Local\Temp\ipykernel_7192\2303233231.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_csv(r'D:\Machine_learning_projects\notebooks\data\UCI_Credit_Card_updated.csv',index_col="ID")
data_train,data_test = train_test_split(data,random_state=42,test_size=.15)
df= data.copy()
data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 30000 entries, 1 to 30000
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   LIMIT_BAL                   30000 non-null  float64
 1   SEX                         30000 non-null  object 
 2   EDUCATION                   30000 non-null  object 
 3   MARRIAGE                    30000 non-null  object 
 4   AGE                         30000 non-null  int64  
 5   PAY_0                       30000 non-null  int64  
 6   PAY_2                       30000 non-null  int64  
 7   PAY_3                       30000 non-null  int64  
 8   PAY_4                       30000 non-null  int64  
 9   PAY_5                       30000 non-null  int64  
 10  PAY_6                       30000 non-null  int64  
 11  BILL_AMT1                   30000 non-null  float64
 12  BILL_AMT2                   30000 non-null  float64
 13  BILL_AMT3                   30000 no

In [3]:
df.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month'],
      dtype='object')

In [4]:
BILL = ['BILL_AMT1', 'BILL_AMT2','BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']
for i in BILL:
    lower_bound = df[i].quantile(0.0)  
    upper_bound = df[i].quantile(1)
    df_filtered = df[(df[i] >= lower_bound) & (df[i] <= upper_bound)]

PAY_AMT = ['PAY_AMT1','PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'] 
for i in PAY_AMT:
    lower_bound = df_filtered[i].quantile(0.0)  
    upper_bound = df_filtered[i].quantile(1)
    df_filtered = df_filtered[(df_filtered[i] >= lower_bound) & (df_filtered[i] <= upper_bound)]

In [5]:
X_train,X_test,Y_train,Y_test= train_test_split(df_filtered.drop(columns=['default.payment.next.month']),df_filtered['default.payment.next.month'],test_size=.2,random_state=42)

In [6]:
df = pd.DataFrame(X_train)
df.head().T

ID,21754,252,22942,619,17091
LIMIT_BAL,80000.0,30000.0,180000.0,60000.0,130000.0
SEX,Female,Male,Female,Male,Female
EDUCATION,University,University,Other,Graduate School,University
MARRIAGE,Single,Single,Married,Single,Single
AGE,24,28,44,25,25
PAY_0,0,0,0,0,0
PAY_2,0,0,0,0,0
PAY_3,0,0,-1,0,0
PAY_4,0,0,-1,0,0
PAY_5,0,0,-1,0,0


In [7]:
#imputation transformer
trf1 = ColumnTransformer(
    [
        ('impute_numerical_columns1',SimpleImputer(strategy='median'),[0]),
        ('impute_categorical_columns1',SimpleImputer(strategy='most_frequent'),[1,2,3]),
        ('impute_numerical_columns2',SimpleImputer(strategy='median'),[4]),
        ('impute_categorical_columns2',SimpleImputer(strategy='most_frequent'),[5,6,7,8,9,10]),
        ('impute_numerical_columns3',SimpleImputer(strategy='median'),[11,12,13,14,15,16,17,18,19,20,21,22])
    ]
                         ,remainder='passthrough')

In [8]:
trf2 = ColumnTransformer(
    [
        ('yao_jhonson_transformation1',PowerTransformer(),[0,4,11,12,13,14,15,16,17,18,19,20,21,22])
        ],remainder="passthrough")

In [9]:
trf3 = ColumnTransformer([
    ('one_hot_encoding1',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[14,15,16,17,18,19,20,21,22])
        ]
    ,remainder='passthrough')

In [10]:
pipe = Pipeline(
    [
        ('trf1',trf1),
        ('trf2',trf2),
        ('trf3',trf3)
    ]
    )

In [11]:
X_train_transformed = pipe.fit_transform(X_train)
X_test_transformed = pipe.transform(X_test)

In [12]:
output = LabelEncoder()
Y_train_transformed =output.fit_transform(Y_train)
Y_test_transformed = output.transform(Y_test)

In [13]:
Y_train_transformed

array([0, 0, 0, ..., 0, 1, 1])

In [14]:
RFC = RandomForestClassifier(n_jobs=-1)
RFC.fit(X_train_transformed,Y_train_transformed)

RandomForestClassifier(n_jobs=-1)

In [14]:
Y_predict = RFC.predict(X_test_transformed)

In [15]:
accuracy_score(Y_test_transformed,Y_predict)

0.8153333333333334

In [16]:
logitic = LogisticRegression(n_jobs= -1)
logitic.fit(X_train_transformed,Y_train_transformed)
Y_predict_logistic = logitic.predict(X_test_transformed)

In [17]:
accuracy_score(Y_test_transformed,Y_predict_logistic)

0.82

**Hyperparameter tunning**

In [18]:
classifier2 = LogisticRegression()
param_grid_lr = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  
    'penalty': ['l1', 'l2','elasticnet'],  
    'max_iter': [100, 200, 300], 
    'solver':['lbfgs','saga']
}

model2 = GridSearchCV(estimator=classifier2, param_grid=param_grid_lr, n_jobs=-1, verbose=3, cv=3)

model2.fit(X_train_transformed, Y_train_transformed)

print("Best Parameters:", model2.best_params_)
print("Best Accuracy:", model2.best_score_)


Fitting 3 folds for each of 108 candidates, totalling 324 fits


d:\Statistics\States\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
162 fits failed out of a total of 324.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Statistics\States\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Statistics\States\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Statistics\States\Lib\site-packages\sklearn\linear_model\_logistic.py", li

Best Parameters: {'C': 1, 'max_iter': 300, 'penalty': 'l1', 'solver': 'saga'}
Best Accuracy: 0.8209583333333333


d:\Statistics\States\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [19]:
# classifier = RandomForestClassifier()
# param_drid = {'min_samples_leaf':[15,20,30], 'n_estimators':[75,100,150,200],'max_depth':[None,5,8], 'criterion':['gini','entropy']}
# model = RandomizedSearchCV(estimator=classifier, param_distributions=param_drid,n_jobs=-1,verbose=10)
# model.fit(X_train,Y_train)

In [20]:
classifier1 = RandomForestClassifier()
param_drid = {'min_samples_leaf':[15,20,30], 'n_estimators':[100,200,300,400],'max_depth':[None,5,8], 'criterion':['gini','entropy']}
model1 = GridSearchCV(estimator=classifier1, param_grid=param_drid,n_jobs=-1,verbose=3,cv = 3)
model1.fit(X_train_transformed,Y_train_transformed)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 8],
                         'min_samples_leaf': [15, 20, 30],
                         'n_estimators': [100, 200, 300, 400]},
             verbose=3)

In [21]:
model1.best_params_

{'criterion': 'gini',
 'max_depth': None,
 'min_samples_leaf': 15,
 'n_estimators': 300}

In [22]:
RFC2 = RandomForestClassifier(n_estimators=300,min_samples_leaf=15,max_depth= None,criterion= 'gini',n_jobs=-1)
RFC2.fit(X_train_transformed,Y_train_transformed)

RandomForestClassifier(min_samples_leaf=15, n_estimators=300, n_jobs=-1)

In [23]:
Y_predict = RFC2.predict(X_test_transformed)
print(accuracy_score(Y_test_transformed,Y_predict))
print(f1_score(Y_test_transformed,Y_predict))
print(recall_score(Y_test_transformed,Y_predict))

0.821
0.4575757575757576
0.345011424219345


In [21]:
random_forest_classifier = RandomForestClassifier(n_estimators=300,min_samples_leaf=15,max_depth= None,criterion= 'gini',n_jobs=-1)
cv_scores = cross_val_score(random_forest_classifier, X_train_transformed, Y_train_transformed, cv=5)  # Adjust cv as needed

print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", cv_scores.mean())
print(f1_score(Y_test_transformed,Y_predict))

Cross-Validation Scores: [0.818125   0.82333333 0.81458333 0.81916667 0.81708333]
Mean Accuracy: 0.8184583333333334


In [24]:
xgb = XGBClassifier(n_jobs = -1)
xgb.fit(X_train_transformed,Y_train_transformed)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [25]:
Y_predict = xgb.predict(X_test_transformed)
accuracy_score(Y_test_transformed,Y_predict)

0.817

In [26]:
classifier2 = XGBClassifier()
param_grid_xgb = {
    'min_child_weight': [1, 5, 10],
    'gamma': [0.5, 1, 1.5, 2, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'max_depth': [3, 4, 5]
}
model2 = GridSearchCV(estimator=classifier2, param_grid=param_grid_xgb, n_jobs=-1, verbose=3, cv=3)
model2.fit(X_train_transformed, Y_train_transformed)


Fitting 3 folds for each of 405 candidates, totalling 1215 fits


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.8, 1.0],
                         'gamma': [0.5, 1, 1.5, 2, 5], 'max_depth': [3, 4, 5],
                         'min_child_weight': [1, 5, 10],
                         'subsample': [0.6, 0.8, 1.0]},
             verbose=3)

In [27]:
model2.best_params_

{'colsample_bytree': 0.8,
 'gamma': 5,
 'max_depth': 4,
 'min_child_weight': 5,
 'subsample': 1.0}

In [28]:
xgb = XGBClassifier(colsample_bytree = 0.8,gamma= 5,max_depth= 4,min_child_weight= 5,subsample= 1.0,n_jobs = -1)
xgb.fit(X_train_transformed,Y_train_transformed)
Y_predict = xgb.predict(X_test_transformed)
accuracy_score(Y_test_transformed,Y_predict)

0.8193333333333334

In [20]:
xgb_classifier = XGBClassifier(colsample_bytree = 0.8,gamma= 5,max_depth= 4,min_child_weight= 5,subsample= 1.0,n_jobs = -1)
cv_scores = cross_val_score(xgb_classifier, X_train_transformed, Y_train_transformed, cv=5)  # Adjust cv as needed

print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", cv_scores.mean())

Cross-Validation Scores: [0.81875    0.82395833 0.81666667 0.81708333 0.82041667]
Mean Accuracy: 0.819375


In [14]:


# Assuming you have X_train_transformed and Y_train_transformed defined

# Specify the GPU device ID based on the information obtained
gpu_device_id = 0  # GPU ID is 0 in this case

# Create XGBoost classifier with GPU support
xgb_classifier = XGBClassifier(
    colsample_bytree=0.8,
    gamma=5,
    max_depth=4,
    min_child_weight=5,
    subsample=1.0,
    n_jobs=-1,
    tree_method='gpu_hist',
    gpu_id=gpu_device_id
)

# Perform cross-validation
cv_scores = cross_val_score(xgb_classifier, X_train_transformed, Y_train_transformed, cv=5)

# Print the results
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", cv_scores.mean())



d:\Machine_learning_projects\env\lib\site-packages\xgboost\core.py:160: UserWarning: [16:23:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
d:\Machine_learning_projects\env\lib\site-packages\xgboost\core.py:160: UserWarning: [16:23:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
d:\Machine_learning_projects\env\lib\site-packages\xgboost\core.py:160: UserWarning: [16:23:49] WA

Cross-Validation Scores: [0.81833333 0.82416667 0.8175     0.81729167 0.81875   ]
Mean Accuracy: 0.8192083333333333


d:\Machine_learning_projects\env\lib\site-packages\xgboost\core.py:160: UserWarning: [16:23:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
d:\Machine_learning_projects\env\lib\site-packages\xgboost\core.py:160: UserWarning: [16:23:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [20]:
xgb_classifier1 = XGBClassifier(
    colsample_bytree=0.8,
    gamma=5,
    max_depth=4,
    min_child_weight=5,
    subsample=1.0,
    n_jobs=-1,
    tree_method='gpu_hist',
    gpu_id=gpu_device_id
)

xgb_classifier1.fit(X_train_transformed,Y_train_transformed)
Y_predict = xgb_classifier1.predict(X_test_transformed)
accuracy_score(Y_test_transformed,Y_predict)

d:\Machine_learning_projects\env\lib\site-packages\xgboost\core.py:160: UserWarning: [16:38:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


0.821

In [21]:
from sklearn.metrics import f1_score,recall_score
print(f1_score(Y_test_transformed,Y_predict))
print(recall_score(Y_test_transformed,Y_predict))

0.4586693548387097
0.3465346534653465


In [29]:
from sklearn.ensemble import StackingClassifier
base_classifiers = [
    ('xgb', XGBClassifier(colsample_bytree = 0.8,gamma= 5,max_depth= 4,min_child_weight= 5,subsample= 1.0,n_jobs = -1,random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=300,min_samples_leaf=15,max_depth= None,criterion= 'gini',n_jobs=-1,random_state=42)),
    ('lr', LogisticRegression(C=1, max_iter= 500,random_state=42,solver = 'saga',penalty='l1'))
]


stacking_classifier = StackingClassifier(estimators=base_classifiers, final_estimator=LogisticRegression(max_iter=500))
stacking_classifier.fit(X_train_transformed, Y_train_transformed)
stacking_predictions = stacking_classifier.predict(X_test_transformed)
accuracy = accuracy_score(Y_test_transformed, stacking_predictions)
print(f"Stacking Classifier Accuracy: {accuracy}")

d:\Statistics\States\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Statistics\States\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Statistics\States\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Statistics\States\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Statistics\States\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Stacking Classifier Accuracy: 0.8201666666666667


d:\Statistics\States\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
stacking_classifier = StackingClassifier(estimators=base_classifiers, final_estimator=LogisticRegression(max_iter=500))
cv_scores = cross_val_score(stacking_classifier, X_train_transformed, Y_train_transformed, cv=3)  # Adjust cv as needed

print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", cv_scores.mean())


d:\Statistics\States\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Statistics\States\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Statistics\States\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Statistics\States\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Statistics\States\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Statistics\States\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means 

Finally XGBosst Classifier is relavently giving good result.

In [15]:
import optuna
from sklearn.model_selection import cross_val_score


def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
    }

    classifier = XGBClassifier(**params)

    # Assuming you have your data (X, y) ready
    # Replace this with your own cross-validation or training logic
    score = cross_val_score(classifier, X_train_transformed, Y_train_transformed, scoring='accuracy', cv=5).mean()

    return score


d:\Machine_learning_projects\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
study = optuna.create_study(direction='maximize')  # or 'minimize' depending on the metric
study.optimize(objective, n_trials=100)  # Adjust the number of trials as needed

# Get the best hyperparameters
best_params = study.best_params
print('Best Hyperparameters:', best_params)


[I 2024-01-31 22:32:14,292] A new study created in memory with name: no-name-ff04005b-7cc1-4d18-b80c-4dfba6f7140a
[I 2024-01-31 22:32:17,123] Trial 0 finished with value: 0.8185 and parameters: {'n_estimators': 137, 'learning_rate': 0.06980365828951579, 'max_depth': 6, 'min_child_weight': 5, 'subsample': 0.6660079724603318, 'colsample_bytree': 0.8666849977130375}. Best is trial 0 with value: 0.8185.
[I 2024-01-31 22:32:19,213] Trial 1 finished with value: 0.819625 and parameters: {'n_estimators': 199, 'learning_rate': 0.04519856270144915, 'max_depth': 5, 'min_child_weight': 2, 'subsample': 0.5617215877013668, 'colsample_bytree': 0.8210795068753531}. Best is trial 1 with value: 0.819625.
[I 2024-01-31 22:32:21,617] Trial 2 finished with value: 0.8188333333333333 and parameters: {'n_estimators': 179, 'learning_rate': 0.05413229453500419, 'max_depth': 7, 'min_child_weight': 3, 'subsample': 0.6516703099316454, 'colsample_bytree': 0.8385752168903994}. Best is trial 1 with value: 0.819625.
[

Best Hyperparameters: {'n_estimators': 136, 'learning_rate': 0.08581386246280566, 'max_depth': 3, 'min_child_weight': 3, 'subsample': 0.715231665064311, 'colsample_bytree': 0.5487455924565161}


In [18]:
best_classifier = XGBClassifier(**best_params)
best_classifier.fit(X_train_transformed, Y_train_transformed)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5487455924565161, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.08581386246280566,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=136, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [32]:
used_params = best_classifier.get_params()
print(used_params)

{'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.5487455924565161, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.08581386246280566, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 3, 'max_leaves': None, 'min_child_weight': 3, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 136, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': 0.715231665064311, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}


In [20]:
XGboost_predictions = best_classifier.predict(X_test_transformed)
accuracy = accuracy_score(Y_test_transformed, XGboost_predictions)
print(f"Stacking Classifier Accuracy: {accuracy}")

Stacking Classifier Accuracy: 0.8195


In [22]:
print(classification_report(Y_test_transformed, XGboost_predictions))

              precision    recall  f1-score   support

           0       0.84      0.95      0.89      4687
           1       0.67      0.35      0.46      1313

    accuracy                           0.82      6000
   macro avg       0.75      0.65      0.68      6000
weighted avg       0.80      0.82      0.80      6000



In [25]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0),
    }

    classifier = RandomForestClassifier(**params,n_jobs=-1)

    # Assuming you have your data (X, y) ready
    # Replace this with your own cross-validation or training logic
    score = cross_val_score(classifier,X_train_transformed, Y_train_transformed, scoring='accuracy', cv=5,n_jobs=-1).mean()

    return score


In [26]:
study1 = optuna.create_study(direction='maximize')  # or 'minimize' depending on the metric
study1.optimize(objective, n_trials=100)  # Adjust the number of trials as needed

# Get the best hyperparameters
best_params = study1.best_params
print('Best Hyperparameters:', best_params)


[I 2024-01-31 22:46:09,038] A new study created in memory with name: no-name-a7268854-72e9-4695-b8fe-c9da1dae192b
[I 2024-01-31 22:46:17,382] Trial 0 finished with value: 0.8184999999999999 and parameters: {'n_estimators': 66, 'max_depth': 17, 'min_samples_split': 4, 'min_samples_leaf': 6, 'max_features': 0.7577790435686723}. Best is trial 0 with value: 0.8184999999999999.
[I 2024-01-31 22:46:28,812] Trial 1 finished with value: 0.8194166666666668 and parameters: {'n_estimators': 120, 'max_depth': 15, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 0.6622209386141391}. Best is trial 1 with value: 0.8194166666666668.
[I 2024-01-31 22:46:35,433] Trial 2 finished with value: 0.8193333333333334 and parameters: {'n_estimators': 154, 'max_depth': 5, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 0.6357442897306094}. Best is trial 1 with value: 0.8194166666666668.
[I 2024-01-31 22:46:41,192] Trial 3 finished with value: 0.8202083333333334 and parameters: {'n_est

Best Hyperparameters: {'n_estimators': 121, 'max_depth': 9, 'min_samples_split': 16, 'min_samples_leaf': 6, 'max_features': 0.29499734960645674}


In [27]:
best_classifier1 = RandomForestClassifier(**best_params,n_jobs=-1)
best_classifier1.fit(X_train_transformed, Y_train_transformed)


RandomForestClassifier(max_depth=9, max_features=0.29499734960645674,
                       min_samples_leaf=6, min_samples_split=16,
                       n_estimators=121, n_jobs=-1)

In [28]:
random_predictions = best_classifier1.predict(X_test_transformed)
accuracy = accuracy_score(Y_test_transformed, random_predictions)
print(f"Stacking Classifier Accuracy: {accuracy}")

Stacking Classifier Accuracy: 0.821


In [30]:
print(classification_report(Y_test_transformed,random_predictions))
print('-------------------------------------------------------------------')
print(classification_report(Y_test_transformed, XGboost_predictions))

              precision    recall  f1-score   support

           0       0.84      0.95      0.89      4687
           1       0.68      0.35      0.46      1313

    accuracy                           0.82      6000
   macro avg       0.76      0.65      0.68      6000
weighted avg       0.80      0.82      0.80      6000

-------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.84      0.95      0.89      4687
           1       0.67      0.35      0.46      1313

    accuracy                           0.82      6000
   macro avg       0.75      0.65      0.68      6000
weighted avg       0.80      0.82      0.80      6000



In [ ]:
objective ='binary:logistic', base_score= None, booster= None, callbacks= None, colsample_bylevel= None, colsample_bynode= None, colsample_bytree= 0.5487455924565161, device= None, early_stopping_rounds= None, enable_categorical= False, eval_metric= None, feature_types= None, gamma= None, importance_type= None, interaction_constraints= None, learning_rate=0.08581386246280566,max_bin= None, max_cat_threshold = None, max_cat_to_onehot= None, max_delta_step= None, max_depth = 3, max_leaves =  None, min_child_weight =  3, missing =  nan, monotone_constraints=None, multi_strategy=None, n_estimators=136, n_jobs=-1, num_parallel_tree=None, random_state=None, reg_alpha= None, reg_lambda=None, sampling_method=None, scale_pos_weight= None, subsample= 0.715231665064311, tree_method= None, validate_parameters= None, verbosity= None